In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
def mountSpeed(az, el):
    # Calculate theoretical drive speed vs Az/El
    t0 = Time('2023-01-01T00:00:00') # doesn't really matter when
    altAz0 = SkyCoord(AltAz(alt=el*u.deg, az=az*u.deg, obstime=t0, location=AUXTEL_LOCATION))
    RaDec = altAz0.transform_to(ICRS)
    t1 = t0 + TimeDelta(1.0, format='sec')
    altAz1 = RaDec.transform_to(AltAz(obstime=t1, location=AUXTEL_LOCATION))
    #print(altAz1)
    newAz = altAz1.az.deg
    if az < 5.0 and newAz > 355.0:
        delta = newAz -360.0 - az
    else:
        delta = newAz - az

    azVel = abs(delta) * 3600.0
    elVel = abs(altAz1.alt.deg - el) * 3600.0
    #print(az, altAz1.az.deg, el, altAz1.alt.deg)
    return(azVel, elVel)
Naz = 100
Nel = 50
azimuths = np.linspace(0, 360, Naz)
els = np.linspace(5, 85, Nel)
azValues = np.zeros([Naz, Nel])
elValues = np.zeros([Naz, Nel])
for i, az in enumerate(azimuths):
    for j, el in enumerate(els):
        [azSpeed, elSpeed] = mountSpeed(az, el)
        azValues[i,j] = azSpeed
        elValues[i,j] = elSpeed
        #print(az, el, mountSpeed(az, el)[0])
    
r, theta = np.meshgrid(els, np.radians(azimuths))


In [ ]:
dates = ["2023-05-09","2023-05-10","2023-05-11","2023-04-25","2023-03-23",\
         "2023-03-01","2023-03-02","2023-03-14","2023-03-15","2023-03-16"]
dates += ["2023-05-23","2023-05-24","2023-07-04","2023-07-05","2023-07-06",\
         "2023-07-18","2023-07-20","2023-08-01","2023-08-02","2023-08-03",\
         "2023-08-15","2023-08-16","2023-08-17","2023-08-29","2023-09-12",\
         "2023-09-13","2023-09-14"]

maxWindSpeed = 6.0
mountErrorLimit = 0.4
alts = []
radAzs = []
for date in dates:
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')
    try:
        wind = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                            ['speed'],  start, end)
        medianWind = np.median(wind.values[0])
    except:
        print("Wind not found")
        continue
    if medianWind > maxWindSpeed or medianWind < 1.0 :
           continue
    # Get Rubin TV JSON
    else:
        print(date, medianWind)
        continue
    filename = f'/scratch/cslage/starTracker/auxtel_{date}.json'
    df = pd.read_json(filename)
    df = df.transpose()
                               
    df_bad = df[df['Mount motion image degradation'] > mountErrorLimit]
    df_good = df[df['Mount motion image degradation'] <= mountErrorLimit]
    mount_degradation = df['Mount motion image degradation']
    total = np.count_nonzero(~pd.isna(mount_degradation.values))

    columns_to_keep = ['Altitude', 'Azimuth', 'Exposure id', 'Mount motion image degradation', 'TAI']
    df_bad = df_bad[columns_to_keep]
    df_bad['Type'] = 'Other'
    df_good = df_good[columns_to_keep]
    print(f"For {date} there are {len(df_bad)} values > {mountErrorLimit} out of {total} values")
    alts += list(df_bad['Altitude'].values)
    azs = df_bad['Azimuth'].values
    radAzs += list(np.radians(azs.astype(float)))
print(len(alts), len(radAzs))

In [ ]:
#-- Plot... ------------------------------------------------
fig=plt.figure(figsize=(16,16))
#plt.subplots_adjust(wspace = 1.0)
vmin = -2.0
vmax = 2.0
levels = np.linspace(vmin, vmax, 9)
ax1 = plt.subplot(121, projection='polar')
ax1.set_title("Log10(Theoretical Azimuth speed (arcseconds/sec))")
ax1.invert_yaxis()
contourf_1 = ax1.contourf(theta, r, np.log10(azValues), levels=levels)
cax1 = fig.add_axes([ax1.get_position().x1+0.01,ax1.get_position().y0,0.02,ax1.get_position().height])
plt.colorbar(contourf_1, cax=cax1)
ax1.scatter([np.pi], [-AUXTEL_LOCATION.lat.deg], color='cyan', marker='x', s=200, label="SCP")
#ax1.scatter(goodRadAzs, goodAlts, color='green', marker='+', label="Mount motion < 0.4")
ax1.scatter(radAzs, alts, color='red', marker='x', label="Mount motion > 0.4")
ax1.legend()
#plt.savefig('/home/craiglagegit/DATA/stiction_vs_speed_20230302/Mount_Fails_Positions_20230302.pdf')